In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [2]:
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [3]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle Rating\n\n5 stars, 111 reviews\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n\nApartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total number of documents: ",len(docs))

Total number of documents:  11


In [9]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings

In [10]:
# Ebeddings using huggingface

huggingface_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  #"BAAI/bge-small-env-v1.5"
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

/home/muhammed-shafeeh/AI_ML/GenAI/.genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-25 09:01:08.267249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740454268.322688  201703 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740454268.339374  201703 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 09:01:08.460952: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in perform

In [11]:
vectorstore = Chroma.from_documents(docs, huggingface_embedding)

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [13]:
retrieved_docs = retriever.invoke("What kind of services they provide?")

In [14]:
len(retrieved_docs)

3

In [15]:
print(retrieved_docs[0].page_content)

Contact Us

Our Clients Say!

Discover firsthand experiences from our valued clients through their heartfelt testimonials. From seamless moves to exceptional service, our customers share how we've made their relocation journey stress-free and rewarding. Explore their stories and see why they trust us with their moves time and again.

Get In Touch

Wollert Victoria

0404922328

victoriaonmove07@gmail.com

Quick Links

About Us Contact Us Our Services Terms & Condition

Photo Gallery

Check us out on Google!

© Victoria On Move 2024, All Right Reserved. Designed By HTML Codex

Home


In [16]:
from langchain_community.llms import Ollama

## load ollama model
llm = Ollama(model="llama3.2")
llm

/tmp/ipykernel_201703/790105878.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


Ollama(model='llama3.2')

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_anser_chain = create_stuff_documents_chain(
    llm,
    prompt=prompt,
    
)
rag_chain = create_retrieval_chain(
    retrieved_docs,
    question_anser_chain,
    
)

In [30]:
# Try this format instead:
response = rag_chain.invoke({"query": "What kind of services they provide?"})



print(response)

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Assuming you have these components defined already
# retriever = your_vector_store.as_retriever()
# llm = your_language_model

# Function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) 

# Create a proper RAG prompt template
prompt = ChatPromptTemplate.from_template("""
Answer the question based on the following context:

{context}

Question: {question}
""")

# Build the RAG chain with the correct structure
rag_chain = {
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": RunnablePassthrough()
} | prompt | llm

# Now this should work:
response = rag_chain.invoke({"question": "What kind of services they provide?"})
print(response)

/tmp/ipykernel_201703/3614907524.py:23: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "context": lambda x: retriever.get_relevant_documents(x["question"]),


Based on the provided context, it appears that Victoria On Move provides a range of services related to moving and relocation, including:

1. Interstate removals (e.g., Melbourne to Sydney, Melbourne to Brisbane)
2. Local removals
3. Packing services
4. Apartment moving
5. Villa moving

Additionally, they seem to emphasize their commitment to providing high-quality, stress-free moving experiences for their customers.


In [34]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))

GPU available: []


2025-02-26 06:32:23.545830: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-02-26 06:32:23.546866: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: muhammed-shafeeh-IdeaPad-Gaming-3-15ARH05
2025-02-26 06:32:23.546987: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: muhammed-shafeeh-IdeaPad-Gaming-3-15ARH05
2025-02-26 06:32:23.547904: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.120.0
2025-02-26 06:32:23.547938: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.120.0
2025-02-26 06:32:23.547947: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.120.0
